# Workflow with CrewAI Flows

This notebook demonstrates CrewAI's **Flow** system for building structured, event-driven workflows. We build a simple game session that uses decorators to handle branching logic.

## What You'll Learn

- How to use the **`Flow`** class to define multi-step workflows
- How **`@start`** marks the entry point of a flow
- How **`@router`** creates conditional branching (like an if/else for workflows)
- How **`@listen`** reacts to specific routing outcomes
- How **Pydantic models** provide typed state management across flow steps

## How Flows Differ from Crews

| Feature | Crew | Flow |
|---------|------|------|
| Execution | LLM-driven agents | Deterministic Python logic |
| Branching | Implicit via task context | Explicit via `@router` |
| State | Task outputs passed as context | Pydantic model shared across steps |
| Use case | Open-ended AI tasks | Structured, predictable pipelines |

## Step 1: Imports

We import `Flow` and its decorators from `crewai.flow.flow`, plus `BaseModel` from Pydantic for typed state management.

In [10]:
import random
import nest_asyncio
from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel

nest_asyncio.apply()  # Allow nested event loops (needed in Jupyter)

## Step 2: Define State Model & Flow

The flow has four methods connected by decorators:

```
@start: begin_start()          → Randomly decide win/lose
          ↓
@router: check_outcome()       → Route to "win" or "lose"
          ↓                ↓
@listen("win"):           @listen("lose"):
  celebrate_win()           console_loss()
```

**`GameSession`** is a Pydantic model that holds the shared state (`player_won`) across all flow steps. The `Flow[GameSession]` generic type ensures type safety.

In [11]:
# State model — shared across all flow steps via self.state
class GameSession(BaseModel):
    player_won: bool = False


class GameFlow(Flow[GameSession]):
    """A simple flow that simulates a game with win/lose branching."""

    @start()  # Entry point — runs first when flow.kickoff() is called
    def begin_start(self):
        print("Starting the game session")
        player_outcome = random.choice([True, False])
        self.state.player_won = player_outcome  # Write to shared state

    @router(begin_start)  # Runs after begin_start; returns a route label
    def check_outcome(self):
        if self.state.player_won:
            return "win"   # Routes to methods listening for "win"
        else:
            return "lose"  # Routes to methods listening for "lose"

    @listen("win")  # Only fires when the router returns "win"
    def celebrate_win(self):
        print("Congratulations!")

    @listen("lose")  # Only fires when the router returns "lose"
    def console_loss(self):
        print("Game Over!")

## Step 3: Run the Flow

Instantiate the flow and call `kickoff()`. Run this cell multiple times — you'll see different outcomes each time because the game result is random!

In [12]:
flow = GameFlow()
result = flow.kickoff()

Flow started with ID: 00a096ee-bf62-4431-a86e-ddc48bdc3469

Starting the game session


╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: begin_start                                                                                            │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Game Over!


╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: console_loss                                                                                           │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: check_outcome                                                                                          │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: console_loss                                                                                           │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── ✅ Flow Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  GameFlow                                                                                                       │
│  ID:                                                                                                            │
│  00a096ee-bf62-4431-a86e-ddc48bdc3469                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🌊 Flow Execution ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name:                                                                                                          │
│  GameFlow                                                                                                       │
│  ID:                                                                                                            │
│  00a096ee-bf62-4431-a86e-ddc48bdc3469                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🌊 Flow Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Started                                                                                                   │
│  Name: GameFlow                                                                                                 │
│  ID: 00a096ee-bf62-4431-a86e-ddc48bdc3469                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
flow.plot()

'/var/folders/b_/0hx_r16d2v712fg4k7y_rh140000gp/T/crewai_flow__owtzw3y/crewai_flow.html'